In [68]:
import pandas as pd

df = pd.read_csv('../data/df_news.csv')

In [69]:
df_ticker = df[df['ticker'] == 'NVDA'].copy()

# Eliminar duplicados en función de ciertas columnas y seleccionar solo esas
df_ticker = df_ticker.drop_duplicates(subset=['title', 'datetime', 'relevance_score', 'ticker_sentiment_score'])[
    ['title', 'datetime', 'relevance_score', 'ticker_sentiment_score']
]

# Calcular ticker_score
df_ticker['ticker_score'] = (df_ticker['relevance_score'] * df_ticker['ticker_sentiment_score']) * 5

# Agregar por datetime sumando ticker_score
df_ticker = df_ticker.groupby('datetime', as_index=False).agg({'ticker_score': 'sum'})

# Ordenar por datetime
df_ticker = df_ticker.sort_values(by='datetime', ascending=True)


In [70]:
news_data = df[['datetime', 'title', 'overall_sentiment_score', 'relevance_score']].drop_duplicates()
news_data[['overall_sentiment_score', 'relevance_score']] = news_data[['overall_sentiment_score', 'relevance_score']].apply(pd.to_numeric, errors='coerce')
global_metrics = news_data.groupby('datetime').agg({
            'overall_sentiment_score': lambda x: round(x.mean(), 6),
            'relevance_score': lambda x: round(x.mean(), 6)
        }).rename(columns={
            'overall_sentiment_score': 'all_news_ossm',
            'relevance_score': 'all_news_rsm'
        }).reset_index()

global_metrics['global_score'] = global_metrics['all_news_ossm'] * global_metrics['all_news_rsm']

In [ ]:
df_fn = pd.merge(global_metrics, df_ticker, on='datetime', how='outer')

df_fn['ticker_score'] = df_fn['ticker_score'].fillna(0)
df_fn['global_score'] = df_fn['global_score'].fillna(0)
